
Predicting Fake News using NLP and Machine Learning 

Reperformance using Data set with fake news

https://towardsdatascience.com/predicting-fake-news-using-nlp-and-machine-learning-scikit-learn-glove-keras-lstm-7bbd557c3443


In [1]:
!pip install contractions

     |████████████████████████████████| 287 kB 7.9 MB/s 
     |████████████████████████████████| 106 kB 41.2 MB/s 


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import rcParams
plt.rcParams['figure.figsize'] = [10,10]
import seaborn as sns
sns.set_theme(style="darkgrid")
from wordcloud import WordCloud

import nltk
from nltk import sent_tokenize
''' Required for Google Colab'''
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
''' Required for Google Colab'''
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize

import contractions
import re
import itertools
import datetime
import time
from collections import Counter
import string

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import rcParams
plt.rcParams['figure.figsize'] = [10,10]
import seaborn as sns
sns.set_theme(style="darkgrid")

import nltk
from nltk import sent_tokenize
''' Required for Google Colab'''
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
''' Required for Google Colab'''
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize
import contractions

import string
import joblib
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

# set seed
np.random.seed(42)

## based on validation data

In [4]:
train_df = pd.read_csv('https://raw.githubusercontent.com/smkerr/COVID-fake-news-detection/main/data/Constraint_Train.csv', header=0)
val_df = pd.read_csv('https://raw.githubusercontent.com/smkerr/COVID-fake-news-detection/main/data/Constraint_Val.csv', header=0)

In [5]:
stopwords = nltk.corpus.stopwords.words('english')

def cleantext(string):
    text = string.lower().split()
    text = " ".join(text)
    text = re.sub(r"http(\S)+",' ',text)    
    text = re.sub(r"www(\S)+",' ',text)
    text = re.sub(r"&",' and ',text)  
    tx = text.replace('&amp',' ')
    text = re.sub(r"[^a-zA-Z]+",' ',text)
    text = text.split()
    text = [w for w in text if not w in stopwords]
    text = " ".join(text)
    return text

In [6]:
train_df['tweet'] = train_df['tweet'].map(lambda x: cleantext(x))
val_df['tweet'] = val_df['tweet'].map(lambda x: cleantext(x))

In [7]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2))
tf_idf_transformer = TfidfTransformer(smooth_idf=True)

#count_vectorizer = CountVectorizer(ngram_range=(1, 2))
#tf_idf_transformer = TfidfTransformer(smooth_idf=False)

#count_vectorizer = CountVectorizer(ngram_range=(1, 1))
#tf_idf_transformer = TfidfTransformer(smooth_idf=True)

# fit and transform train data to count vectorizer
count_vectorizer.fit(train_df['tweet'].values)
count_vect_train = count_vectorizer.transform(train_df['tweet'].values)

# fit the counts vector to tfidf transformer
tf_idf_transformer.fit(count_vect_train)
tf_idf_train = tf_idf_transformer.transform(count_vect_train)

# Transform the test data as well
count_vect_test = count_vectorizer.transform(val_df['tweet'].values)
tf_idf_test = tf_idf_transformer.transform(count_vect_test)

In [8]:
def get_perf_metrics(model, i):
    # model name
    model_name = type(model).__name__
    # time keeping
    start_time = time.time()
    print("Training {} model...".format(model_name))
    # Fitting of model
    model.fit(tf_idf_train, train_df["label"])
    print("Completed {} model training.".format(model_name))
    elapsed_time = time.time() - start_time
    # Time Elapsed
    print("Time elapsed: {:.2f} s.".format(elapsed_time))
    # Predictions
    y_pred = model.predict(tf_idf_test)
    # Add to ith row of dataframe - metrics
    df_perf_metrics.loc[i] = [
        model_name,
        model.score(tf_idf_train, train_df["label"]),
        model.score(tf_idf_test, val_df["label"]),
        precision_score(val_df["label"], y_pred, pos_label="real"),
        recall_score(val_df["label"], y_pred, pos_label="real"),
        f1_score(val_df["label"], y_pred, pos_label="real"), "{:.2f}".format(elapsed_time)
    ]
    # keep a track of trained models
    models_trained_list.append(model)
    print("Completed {} model's performance assessment.".format(model_name))

In [9]:
models_list = [LogisticRegression(),
               MultinomialNB(),
               RandomForestClassifier(),
               DecisionTreeClassifier(),
               GradientBoostingClassifier(),
               AdaBoostClassifier(),
               LinearSVC()]

In [10]:
df_perf_metrics = pd.DataFrame(columns=['Model', 'Accuracy_Training_Set', 'Accuracy_Test_Set', 'Precision', 'Recall', 'f1_score'])

In [11]:
df_perf_metrics = pd.DataFrame(columns=[
    'Model', 'Accuracy_Training_Set', 'Accuracy_Test_Set', 'Precision',
    'Recall', 'f1_score', 'Training Time (secs'
])

In [12]:
models_trained_list = []

In [13]:
%%time
for n, model in enumerate(models_list):
    get_perf_metrics(model, n)

Training LogisticRegression model...
Completed LogisticRegression model training.
Time elapsed: 0.75 s.
Completed LogisticRegression model's performance assessment.
Training MultinomialNB model...
Completed MultinomialNB model training.
Time elapsed: 0.03 s.
Completed MultinomialNB model's performance assessment.
Training RandomForestClassifier model...
Completed RandomForestClassifier model training.
Time elapsed: 7.91 s.
Completed RandomForestClassifier model's performance assessment.
Training DecisionTreeClassifier model...
Completed DecisionTreeClassifier model training.
Time elapsed: 2.37 s.
Completed DecisionTreeClassifier model's performance assessment.
Training GradientBoostingClassifier model...
Completed GradientBoostingClassifier model training.
Time elapsed: 8.38 s.
Completed GradientBoostingClassifier model's performance assessment.
Training AdaBoostClassifier model...
Completed AdaBoostClassifier model training.
Time elapsed: 2.11 s.
Completed AdaBoostClassifier model's p

## result

ngram_range=(1, 1)
smooth_idf=True

In [14]:
df_perf_metrics

,Model,Accuracy_Training_Set,Accuracy_Test_Set,Precision,Recall,f1_score,Training Time (secs
0,LogisticRegression,0.958100,0.920561,0.932605,0.914286,0.923354,0.29
1,MultinomialNB,0.953427,0.908411,0.885000,0.948214,0.915517,0.02
2,RandomForestClassifier,0.999844,0.904206,0.927171,0.886607,0.906435,7.50
3,DecisionTreeClassifier,1.000000,0.844860,0.842014,0.866071,0.853873,1.70
4,GradientBoostingClassifier,0.893146,0.860748,0.905325,0.819643,0.860356,6.60
5,AdaBoostClassifier,0.867757,0.844393,0.858053,0.841964,0.849932,3.04
6,LinearSVC,0.997664,0.932710,0.938849,0.932143,0.935484,0.05


ngram_range=(1, 2)
smooth_idf=False

In [28]:
df_perf_metrics

,Model,Accuracy_Training_Set,Accuracy_Test_Set,Precision,Recall,f1_score,Training Time (secs
0,LogisticRegression,0.973209,0.918224,0.920748,0.923214,0.921979,0.67
1,MultinomialNB,0.985514,0.907477,0.869391,0.968750,0.916385,0.02
2,RandomForestClassifier,0.999844,0.907477,0.939885,0.879464,0.908672,8.42
3,DecisionTreeClassifier,1.000000,0.827570,0.822872,0.854464,0.838371,2.76
4,GradientBoostingClassifier,0.894237,0.855607,0.899507,0.815179,0.855269,9.25
5,AdaBoostClassifier,0.865888,0.848598,0.861162,0.847321,0.854185,2.29
6,LinearSVC,1.000000,0.937383,0.931699,0.950000,0.940760,0.05


ngram_range=(1, 2)
smooth_idf=True

In [14]:
df_perf_metrics

,Model,Accuracy_Training_Set,Accuracy_Test_Set,Precision,Recall,f1_score,Training Time (secs
0,LogisticRegression,0.971963,0.917290,0.919858,0.922321,0.921088,0.75
1,MultinomialNB,0.983178,0.908411,0.870192,0.969643,0.917230,0.03
2,RandomForestClassifier,1.000000,0.898598,0.935391,0.866071,0.899397,7.91
3,DecisionTreeClassifier,1.000000,0.836916,0.829206,0.866964,0.847665,2.37
4,GradientBoostingClassifier,0.892835,0.854673,0.904096,0.808036,0.853371,8.38
5,AdaBoostClassifier,0.865576,0.847664,0.864220,0.841071,0.852489,2.11
6,LinearSVC,1.000000,0.937383,0.932456,0.949107,0.940708,0.05


ngram_range=(1, 3)
smooth_idf=False

In [62]:
df_perf_metrics

,Model,Accuracy_Training_Set,Accuracy_Test_Set,Precision,Recall,f1_score,Training Time (secs
0,LogisticRegression,0.981620,0.908411,0.903846,0.923214,0.913428,1.20
1,MultinomialNB,0.994393,0.911682,0.873896,0.971429,0.920085,0.03
2,RandomForestClassifier,1.000000,0.893458,0.946000,0.844643,0.892453,14.86
3,DecisionTreeClassifier,1.000000,0.821963,0.807149,0.866964,0.835988,5.47
4,GradientBoostingClassifier,0.893925,0.855607,0.905095,0.808929,0.854314,19.27
5,AdaBoostClassifier,0.871184,0.849065,0.867281,0.840179,0.853515,3.88
6,LinearSVC,1.000000,0.933645,0.923010,0.952679,0.937610,0.06


Export data frame to latex

In [16]:
df_perf_metrics[["Model","f1_score"]]

,Model,f1_score
0,LogisticRegression,0.923354
1,MultinomialNB,0.915517
2,RandomForestClassifier,0.906435
3,DecisionTreeClassifier,0.853873
4,GradientBoostingClassifier,0.860356
5,AdaBoostClassifier,0.849932
6,LinearSVC,0.935484


In [17]:
print(df_perf_metrics[["Model","f1_score"]].to_latex(index=False))

\begin{tabular}{lr}
\toprule
                     Model &  f1\_score \\
\midrule
        LogisticRegression &  0.923354 \\
             MultinomialNB &  0.915517 \\
    RandomForestClassifier &  0.906435 \\
    DecisionTreeClassifier &  0.853873 \\
GradientBoostingClassifier &  0.860356 \\
        AdaBoostClassifier &  0.849932 \\
                 LinearSVC &  0.935484 \\
\bottomrule
\end{tabular}

